In [2]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from joblib import parallel_backend

df_combats = pd.read_csv("Datos/combats.csv")
df_combats = df_combats.sample(frac = 1)
df_pokemon = pd.read_csv("Datos/pokemon2.csv")
df_pokemon = df_pokemon.sample(frac = 1)

In [3]:
# Limpieza y configuración de dataframes
# Se cambia el nombre de ambos IDs
df_combats.rename(columns = {'First_pokemon':'ID1', 'Second_pokemon':'ID2', 'Winner': 'isFirstWinner'}, inplace = True)
df_combats.loc[df_combats['isFirstWinner'] == df_combats['ID1'], 'isFirstWinner'] = True
df_combats.loc[df_combats['isFirstWinner'] == df_combats['ID2'], 'isFirstWinner'] = False
df_combats.dropna(inplace=True)

In [4]:
df_combats.columns

Index(['ID1', 'ID2', 'isFirstWinner'], dtype='object')

In [5]:
df_pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [6]:
types1 = pd.get_dummies(df_pokemon["Type 1"])
types2 = pd.get_dummies(df_pokemon["Type 2"])
pd.set_option('max_rows', None)
pd.reset_option('max_rows')
types = types1 | types2


df_clean = pd.merge(df_pokemon, types, left_index=True, right_index=True)

df_clean.dropna(thresh=20,inplace=True)
df_clean.drop(["Type 1", 'Type 2'], axis = 1,inplace= True)
df_clean

,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
536,537,Mow Rotom,50,65,107,105,107,86,4,False,...,0,1,0,0,0,0,0,0,0,0
24,25,Rattata,30,56,35,25,35,72,1,False,...,0,0,0,0,1,0,0,0,0,0
754,755,Swirlix,62,48,66,59,57,49,6,False,...,0,0,0,0,0,0,0,0,0,0
194,195,Flaaffy,70,55,55,80,60,45,2,False,...,0,0,0,0,0,0,0,0,0,0
273,274,Grovyle,50,65,45,85,65,95,3,False,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,510,Snover,60,62,50,62,60,40,4,False,...,0,1,0,1,0,0,0,0,0,0
52,53,Parasect,60,95,80,60,80,30,1,False,...,0,1,0,0,0,0,0,0,0,0
114,115,Hitmonlee,50,120,53,35,110,87,1,False,...,0,0,0,0,0,0,0,0,0,0
452,453,Roserade,60,70,65,125,105,90,4,False,...,0,1,0,0,0,1,0,0,0,0


In [7]:
# Variable respuesta, combates en los que el primer pokemon gana
Y = df_combats["isFirstWinner"] == True

In [8]:
# Mapa de correlaciones
corr = round (df_clean.corr(), 3)
corr.style.background_gradient()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
#,1.000000,0.098000,0.103000,0.095000,0.089000,0.086000,0.012000,0.983000,0.154000,-0.040000,0.099000,0.141000,0.016000,0.050000,0.048000,-0.009000,-0.033000,0.169000,0.044000,-0.047000,0.071000,-0.053000,-0.215000,0.003000,0.025000,0.093000,-0.102000
HP,0.098000,1.000000,0.422000,0.240000,0.362000,0.379000,0.176000,0.059000,0.274000,-0.154000,0.010000,0.138000,-0.061000,-0.007000,0.055000,0.010000,0.031000,-0.062000,-0.046000,0.075000,0.082000,0.112000,-0.076000,0.027000,-0.036000,-0.043000,0.030000
Attack,0.103000,0.422000,1.000000,0.439000,0.396000,0.264000,0.381000,0.051000,0.345000,-0.070000,0.143000,0.213000,-0.075000,-0.123000,0.211000,0.046000,0.014000,-0.016000,-0.063000,0.127000,0.021000,-0.075000,-0.073000,-0.070000,0.101000,0.108000,-0.071000
Defense,0.095000,0.240000,0.439000,1.000000,0.224000,0.511000,0.015000,0.042000,0.246000,-0.028000,-0.024000,0.106000,-0.061000,-0.025000,0.002000,-0.035000,-0.068000,0.061000,-0.007000,0.134000,0.020000,-0.175000,-0.094000,0.011000,0.298000,0.351000,0.006000
Sp. Atk,0.089000,0.362000,0.396000,0.224000,1.000000,0.506000,0.473000,0.036000,0.449000,-0.185000,0.037000,0.194000,0.127000,0.026000,-0.048000,0.182000,0.041000,0.035000,0.001000,-0.107000,0.070000,-0.184000,-0.044000,0.237000,-0.106000,-0.006000,0.026000
Sp. Def,0.086000,0.379000,0.264000,0.511000,0.506000,1.000000,0.259000,0.028000,0.364000,-0.082000,-0.021000,0.139000,0.017000,0.093000,0.014000,0.026000,-0.009000,0.045000,-0.005000,-0.077000,0.060000,-0.112000,-0.047000,0.190000,0.019000,0.103000,-0.025000
Speed,0.012000,0.176000,0.381000,0.015000,0.473000,0.259000,1.000000,-0.023000,0.327000,-0.071000,0.068000,0.123000,0.130000,-0.098000,0.076000,0.073000,0.237000,-0.061000,-0.096000,-0.096000,-0.005000,0.052000,-0.030000,0.108000,-0.165000,-0.098000,-0.049000
Generation,0.983000,0.059000,0.051000,0.042000,0.036000,0.028000,-0.023000,1.000000,0.080000,-0.019000,0.094000,0.102000,0.006000,0.066000,0.051000,0.006000,-0.036000,0.156000,0.073000,-0.051000,0.038000,-0.036000,-0.192000,-0.022000,0.009000,0.079000,-0.105000
Legendary,0.154000,0.274000,0.345000,0.246000,0.449000,0.364000,0.327000,0.080000,1.000000,-0.094000,-0.021000,0.226000,0.018000,-0.005000,-0.006000,0.047000,0.094000,-0.014000,-0.067000,-0.007000,0.041000,-0.086000,-0.086000,0.169000,-0.013000,0.019000,-0.066000
Bug,-0.040000,-0.154000,-0.070000,-0.028000,-0.185000,-0.082000,-0.071000,-0.019000,-0.094000,1.000000,-0.082000,-0.081000,-0.045000,-0.072000,-0.049000,-0.061000,0.065000,-0.059000,-0.034000,-0.064000,-0.070000,-0.120000,0.121000,-0.112000,-0.004000,0.047000,-0.124000


In [9]:
# Igualamos el nombre de la columna del id a ID1
df_clean.rename(columns = {'#':'ID1'}, inplace= True)
#Añadimos todos los campos del 1er pokemon a la tabla
df_combats = df_combats.merge(df_clean, on='ID1')

# Igualamos el nombre de la columna del id a ID2
df_clean.rename(columns = {'ID1':'ID2'}, inplace= True)
#Añadimos todos los campos del 1er pokemon a la tabla
df_combats = df_combats.merge(df_clean, on='ID2')
df_combats.columns

Index(['ID1', 'ID2', 'isFirstWinner', 'Name_x', 'HP_x', 'Attack_x',
       'Defense_x', 'Sp. Atk_x', 'Sp. Def_x', 'Speed_x', 'Generation_x',
       'Legendary_x', 'Bug_x', 'Dark_x', 'Dragon_x', 'Electric_x', 'Fairy_x',
       'Fighting_x', 'Fire_x', 'Flying_x', 'Ghost_x', 'Grass_x', 'Ground_x',
       'Ice_x', 'Normal_x', 'Poison_x', 'Psychic_x', 'Rock_x', 'Steel_x',
       'Water_x', 'Name_y', 'HP_y', 'Attack_y', 'Defense_y', 'Sp. Atk_y',
       'Sp. Def_y', 'Speed_y', 'Generation_y', 'Legendary_y', 'Bug_y',
       'Dark_y', 'Dragon_y', 'Electric_y', 'Fairy_y', 'Fighting_y', 'Fire_y',
       'Flying_y', 'Ghost_y', 'Grass_y', 'Ground_y', 'Ice_y', 'Normal_y',
       'Poison_y', 'Psychic_y', 'Rock_y', 'Steel_y', 'Water_y'],
      dtype='object')

In [10]:
corr = round (df_combats.corr(), 3)
corr.style.background_gradient()

,ID1,ID2,HP_x,Attack_x,Defense_x,Sp. Atk_x,Sp. Def_x,Speed_x,Generation_x,Legendary_x,Bug_x,Dark_x,Dragon_x,Electric_x,Fairy_x,Fighting_x,Fire_x,Flying_x,Ghost_x,Grass_x,Ground_x,Ice_x,Normal_x,Poison_x,Psychic_x,Rock_x,Steel_x,Water_x,HP_y,Attack_y,Defense_y,Sp. Atk_y,Sp. Def_y,Speed_y,Generation_y,Legendary_y,Bug_y,Dark_y,Dragon_y,Electric_y,Fairy_y,Fighting_y,Fire_y,Flying_y,Ghost_y,Grass_y,Ground_y,Ice_y,Normal_y,Poison_y,Psychic_y,Rock_y,Steel_y,Water_y
ID1,1.000000,0.001000,0.108000,0.108000,0.087000,0.093000,0.080000,0.013000,0.983000,0.154000,-0.047000,0.095000,0.146000,0.024000,0.054000,0.045000,-0.004000,-0.032000,0.158000,0.033000,-0.048000,0.066000,-0.039000,-0.218000,0.003000,0.021000,0.091000,-0.102000,0.003000,-0.002000,0.001000,0.007000,0.001000,-0.001000,0.001000,0.002000,-0.008000,-0.001000,-0.001000,-0.002000,-0.007000,-0.006000,-0.001000,-0.001000,-0.006000,0.007000,-0.001000,0.005000,-0.002000,-0.000000,-0.000000,-0.001000,0.002000,0.013000
ID2,0.001000,1.000000,0.006000,0.006000,0.009000,0.009000,0.010000,0.004000,-0.001000,-0.000000,-0.002000,-0.002000,0.001000,0.008000,-0.005000,0.001000,-0.007000,0.009000,-0.001000,0.006000,0.001000,0.002000,-0.002000,-0.005000,0.004000,0.003000,0.004000,-0.003000,0.113000,0.105000,0.094000,0.087000,0.081000,0.004000,0.983000,0.148000,-0.034000,0.100000,0.138000,0.020000,0.063000,0.049000,-0.017000,-0.051000,0.159000,0.040000,-0.046000,0.068000,-0.046000,-0.215000,0.005000,0.022000,0.099000,-0.104000
HP_x,0.108000,0.006000,1.000000,0.429000,0.247000,0.367000,0.382000,0.188000,0.068000,0.279000,-0.157000,0.012000,0.142000,-0.062000,-0.021000,0.048000,0.015000,0.037000,-0.060000,-0.041000,0.076000,0.084000,0.112000,-0.077000,0.026000,-0.040000,-0.036000,0.025000,0.006000,-0.001000,0.004000,0.004000,0.007000,0.001000,0.008000,0.000000,-0.003000,-0.004000,0.006000,0.002000,-0.008000,0.000000,-0.000000,-0.001000,-0.002000,0.008000,-0.001000,0.000000,-0.001000,-0.001000,0.003000,-0.003000,-0.002000,-0.006000
Attack_x,0.108000,0.006000,0.429000,1.000000,0.437000,0.396000,0.266000,0.386000,0.056000,0.349000,-0.074000,0.140000,0.216000,-0.076000,-0.126000,0.213000,0.047000,0.013000,-0.007000,-0.064000,0.124000,0.011000,-0.073000,-0.077000,-0.064000,0.094000,0.120000,-0.068000,0.002000,-0.005000,-0.000000,-0.004000,-0.001000,-0.003000,0.008000,-0.006000,-0.004000,0.001000,-0.008000,-0.001000,0.002000,-0.003000,-0.000000,-0.003000,-0.003000,0.000000,-0.003000,-0.002000,0.004000,-0.001000,0.004000,-0.004000,-0.004000,0.004000
Defense_x,0.087000,0.009000,0.247000,0.437000,1.000000,0.211000,0.505000,0.007000,0.034000,0.236000,-0.031000,-0.021000,0.102000,-0.065000,-0.022000,-0.001000,-0.040000,-0.068000,0.057000,-0.007000,0.135000,0.013000,-0.169000,-0.096000,0.002000,0.308000,0.363000,0.003000,0.008000,-0.001000,-0.001000,0.003000,0.001000,0.000000,0.010000,-0.008000,-0.003000,0.004000,-0.007000,-0.002000,0.006000,0.000000,-0.007000,-0.002000,-0.005000,0.003000,-0.006000,0.005000,-0.000000,-0.005000,0.008000,0.009000,-0.004000,-0.002000
Sp. Atk_x,0.093000,0.009000,0.367000,0.396000,0.211000,1.000000,0.498000,0.469000,0.039000,0.447000,-0.190000,0.041000,0.191000,0.120000,0.024000,-0.049000,0.183000,0.041000,0.042000,0.003000,-0.103000,0.074000,-0.182000,-0.044000,0.241000,-0.111000,-0.010000,0.019000,0.006000,-0.003000,0.003000,0.002000,0.004000,-0.001000,0.010000,0.002000,0.003000,0.001000,-0.011000,-0.001000,0.002000,0.001000,-0.005000,-0.001000,0.003000,0.004000,-0.005000,0.001000,0.003000,0.003000,0.001000,0.003000,-0.001000,-0.004000
Sp. Def_x,0.080000,0.010000,0.382000,0.266000,0.505000,0.498000,1.000000,0.253000,0.022000,0.350000,-0.078000,-0.011000,0.130000,0.014000,0.100000,0.017000,0.022000,-0.009000,0.039000,-0.003000,-0.071000,0.070000,-0.108000,-0.042000,0.187000,0.017000,0.099000,-0.039000,0.007000,-0.002000,-0.000000,0.001000,-0.001000,0.002000,0.011000,-0.001000,-0.000000,-0.003000,-0.001000,0.004000,0.001000,-0.001000,-0.004000,0.001000,-0.003000,0.005000,-0.001000

In [11]:
X = pd.get_dummies(data = df_combats[['ID1', 'ID2', 'isFirstWinner', 'Name_x', 'HP_x', 'Attack_x',
       'Defense_x', 'Sp. Atk_x', 'Sp. Def_x', 'Speed_x', 'Generation_x',
       'Legendary_x', 'Bug_x', 'Dark_x', 'Dragon_x', 'Electric_x', 'Fairy_x',
       'Fighting_x', 'Fire_x', 'Flying_x', 'Ghost_x', 'Grass_x', 'Ground_x',
       'Ice_x', 'Normal_x', 'Poison_x', 'Psychic_x', 'Rock_x', 'Steel_x',
       'Water_x', 'Name_y', 'HP_y', 'Attack_y', 'Defense_y', 'Sp. Atk_y',
       'Sp. Def_y', 'Speed_y', 'Generation_y', 'Legendary_y', 'Bug_y',
       'Dark_y', 'Dragon_y', 'Electric_y', 'Fairy_y', 'Fighting_y', 'Fire_y',
       'Flying_y', 'Ghost_y', 'Grass_y', 'Ground_y', 'Ice_y', 'Normal_y',
       'Poison_y', 'Psychic_y', 'Rock_y', 'Steel_y', 'Water_y']])

In [12]:
X.head()

,ID1,ID2,HP_x,Attack_x,Defense_x,Sp. Atk_x,Sp. Def_x,Speed_x,Generation_x,Legendary_x,...,Name_y_Zangoose,Name_y_Zapdos,Name_y_Zebstrika,Name_y_Zekrom,Name_y_Zigzagoon,Name_y_Zoroark,Name_y_Zorua,Name_y_Zubat,Name_y_Zweilous,Name_y_Zygarde Half Forme
0,105,593,60,48,45,43,90,42,1,False,...,0,0,0,0,0,0,0,0,0,0
1,751,593,60,150,50,150,50,60,6,False,...,0,0,0,0,0,0,0,0,0,0
2,77,593,65,90,50,85,45,55,1,False,...,0,0,0,0,0,0,0,0,0,0
3,331,593,50,70,100,40,40,30,3,False,...,0,0,0,0,0,0,0,0,0,0
4,522,593,65,110,130,60,65,95,4,False,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [14]:
# Ajustar el modelo Bermoulli
clf = BernoulliNB()
clf.fit(X, Y)
Y_pred = clf.predict(X)
np.mean(Y == Y_pred)

0.57328

In [15]:
# Ajustar el modelo Multinomial
clf = MultinomialNB()
clf.fit(X, Y)
Y_pred = clf.predict(X)
np.mean(Y == Y_pred)

0.57304

In [16]:
# Ajustar el modelo Gausiiano
clf = GaussianNB()
clf.fit(X, Y)
Y_pred = clf.predict(X)
np.mean(Y == Y_pred)

0.56978